# LangChain으로 영상 내용 요약하기 - YouTube

- LangChain을 이용하여 정보량이 쏟아지는 현대 사회에서, **자막이 있는 YouTube 영상**에 대해 자동으로 번역 및 요약하고, 궁금한 내용에 대해 바로 답변을 받아보는 YouTube QA 봇을 만들어봅시다.

### 자료 출처: YTN 뉴스, Apple
- 한글 자막 영상 링크: https://www.youtube.com/watch?v=p-zSbaYJBOs
- 영어 자막 영상 링크: https://www.youtube.com/watch?v=TX9qSaGXFyg

## 발급 받은 OpenAI key를 불러옵니다.

In [ ]:
import os

os.environ["OPENAI_API_KEY"] = ''

## 필요한 모듈을 설치합니다.

In [2]:
!pip3 install  youtube-transcript-api pytube google-search-results googletrans==3.1.0a0 httpx==0.13.3

Defaulting to user installation because normal site-packages is not writeable
  Preparing metadata (setup.py) ... done
  Using cached httpx-0.13.3-py3-none-any.whl.metadata (25 kB)
  Using cached httpcore-0.9.1-py3-none-any.whl.metadata (4.6 kB)
  Using cached h11-0.9.0-py2.py3-none-any.whl.metadata (8.1 kB)
Using cached httpx-0.13.3-py3-none-any.whl (55 kB)
Using cached httpcore-0.9.1-py3-none-any.whl (42 kB)
Using cached h11-0.9.0-py2.py3-none-any.whl (53 kB)
  Created wheel for google-search-results: filename=google_search_results-2.4.2-py3-none-any.whl size=32004 sha256=928b908a6f285eb02628248968b5561993dd79649ac0b7714fbe4ea99a1df775
  Stored in directory: /home/elicer/.cache/pip/wheels/d3/b2/c3/03302d12bb44a2cdff3c9371f31b72c0c4e84b8d2285eeac53
Successfully built google-search-results
  Attempting uninstall: h11
    Found existing installation: h11 0.14.0
    Uninstalling h11-0.14.0:
      Successfully uninstalled h11-0.14.0
  Attempting uninstall: httpcore
    Found existing inst

## YouTube 영상 링크를 주면 자막 텍스트 데이터를 불러오는 함수입니다.
- 언어의 종류에 따라 langage를 영어면 'en', 한글이면 'ko' 등으로 설정해야 합니다.
- add_video_info=True로 하면 썸네일 링크를 포함한 여러 meta data들을 함께 볼 수 있습니다.

In [3]:
from langchain.document_loaders import YoutubeLoader

def get_script(url, language="en", add_video_info=True):
    loader = YoutubeLoader.from_youtube_url(
    url,
    add_video_info=add_video_info,
    language=language,
    )
    return loader.load()

## 2023.03.09일자 "[자막뉴스] 이재용, 제대로 칼 갈았다…최고 기술 꺼내 '승부수' / YTN" 영상의 자막을 추출합니다.

In [8]:
url = "https://www.youtube.com/watch?v=p-zSbaYJBOs"


In [10]:
raw_script_data = get_script(url, 'ko')[0].page_content
raw_script_data

'패널 수요일과 수익성이 좋지 않아 국내에서 oled tv 사업을 철수했던 삼성전자가 10년만에 신제품을 출시했습니다 OLED 기술의 장점을 유지하면서도 밝기 성능을 대폭 개선한게 특징입니다 기존의 주력했던 qled tv도 8k 영상의 초고화질 98인치 크기의 초대형 신제품을 내놨습니다 2023년 삼성전자는 전 세계 프리미엄 TV 시장을 이끌고 있는 네오큐헬리드 신제품들과 삼성 tv의 기술력이 완성한 삼성 oled를 통해 고객들에게 최상의 스크린 경험을 제공할 것입니다 올레드 TV 사업 10주년에 맞은 lg전자도 삼성전자보다 하루 앞서 신제품을 공개했습니다 기존 제품보다 70% 더 밝아진 것부터 세계 최초로 4K 영상을 무선 전송해 복잡한 주변 연결선을 없앤 TV 등 29개 모델을 선보였습니다 도전장에는 삼성전자를 향해선 그동안 집약된 기술력을 토대로 자신감을 내비쳤습니다 굉장히 편리한 사용성을 제공하고 최근에 제공해드린 그 무선 기술까지 이런 기술을 가지고 저희가 향후에도 올레드 시장을 리딩할 수 있다고 생각을 하고 있습니다 삼성전자가 기존 사업계획까지 번복하고 OLED 시장에 뛰어든 건 최근 빠르게 늘어나고 있는 수요 때문입니다 2013년 4000대 수준이었던 출하량은 10년 사이 1800배 매출액은 280배 넘게 늘었고 프리미엄 시장에서도 OLED tv가 차지하는 비중이 절반에 달합니다 lg전자가 주도권을 가지고 있던 oled tv 시장에 삼성전자가 뛰어들면서 두 기업이 10년 만에 맞붙게 됐는데 치열한 경쟁 속에 제품 가격 하락에 대한 기대감도 커지고 있습니다 lg전자 올레드 TV 국내 출고가는 크기에 따라 300에서 900만원 선으로 다양한데 후발주자인 삼성전자는 lg전자의 중간 수준으로 가격대를 낮춰 공격적인 마케팅을 펼치고 있습니다 tcl가 화웨이 샤오미 등 저가 물량공세를 펼치는 중국 업체들의 추격도 매서운만큼 점유율을 늘리기 위한 업체들의 경쟁은 더 치열해질 것으로 보입니다 YTN 뉴넬입니다 [음악]'

## 뉴스 영상 스크립트 요약하기
- 이전 실습에서 사용했던 qa_bot 함수를 그대로 사용합니다.

In [12]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.question_answering import load_qa_chain
from langchain.chains import AnalyzeDocumentChain
from langchain.callbacks import get_openai_callback
import FinanceDataReader as fdr
import time

def qa_bot(source, question, model='gpt-3.5-turbo', temperature=0, chain_type="map_reduce"):

    exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]

    model = ChatOpenAI(model=model, temperature=0)

    qa_chain = load_qa_chain(model, chain_type=chain_type)
    qa_document_chain = AnalyzeDocumentChain(combine_docs_chain=qa_chain)

    with get_openai_callback() as cb:

        start = time.time()
        print(qa_document_chain.run(input_document=source, question=question))
        end = time.time()

        print(f"Total Tokens: {cb.total_tokens}")
        print(f"Prompt Tokens: {cb.prompt_tokens}")
        print(f"Completion Tokens: {cb.completion_tokens}")
        print(f"Total Cost (USD): {cb.total_cost:.2f}$, Total Cost (Won): {cb.total_cost*exchange_rate:.2f}₩")
        print(f"걸린 시간: {end-start:.2f}초")

In [14]:
qa_bot(raw_script_data, question=input())

/tmp/ipykernel_19160/1267335159.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


| 브랜드     | 모델명     | 특징                                                                                   |
|------------|------------|----------------------------------------------------------------------------------------|
| 삼성전자   | OLED TV    | 밝기 성능 대폭 개선, 8K 영상 지원, 98인치 초대형 신제품, 네오큐헬리드 기술 적용         |
Total Tokens: 1527
Prompt Tokens: 1255
Completion Tokens: 272
Total Cost (USD): 0.00$, Total Cost (Won): 3.28₩
걸린 시간: 4.09초


## 뉴스 영상에서 궁금한 내용 질문하기

## 영어 자막이 있는 Apple의 2023.06.06일자 "Introducing Apple Vision Pro" 영상의 자막을 추출합니다.

In [15]:
url = "https://www.youtube.com/watch?v=TX9qSaGXFyg"

raw_script_data = get_script(url, 'en')[0].page_content
raw_script_data

'♪ "Let\'s Live"\nby FKJ ♪ ♪ Introducing Apple Vision Pro. The era of spatial computing is here. ♪ ♪ Serene electronic music ♪ When you put on\nApple Vision Pro, you see your world\nand everything in it. Your favorite apps\nlive right in front of you, but now they\'re in your space. This is visionOS, Apple\'s first-ever\nspatial operating system. It\'s familiar,\nyet groundbreaking. You navigate with your eyes. Simply tap to select, flick to scroll, and use your voice to dictate. It\'s like magic. Apps have dimension, react to light, and cast shadows. Even though\nthese spatial experiences are happening inside Vision Pro, it looks, sounds, and feels\nlike they are physically there. [LAUGHTER] Foundational to Apple Vision Pro is that you\'re not isolated\nfrom other people. When someone else\nis in the room, you can see them\nand they can see you. - Don\'t even talk to me about --\nSushi? - Sushi.\n- All right, see you. - Take care. - Because you\'re not limited\nby a display, apps live

## 자막 내용 번역하기
- 이전 실습에서 사용했던 translator 함수를 그대로 사용합니다.

In [16]:
import googletrans

def translator(eng_sent):
    translator = googletrans.Translator()
    result = translator.translate(eng_sent, dest='ko')
    return result.text

In [18]:
kr_row_data = translator(raw_script_data)

## Apple 영상 스크립트 요약하기

In [19]:
qa_bot(kr_row_data, question= input())

/tmp/ipykernel_19160/1267335159.py:10: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  exchange_rate = fdr.DataReader('USD/KRW').iloc[-1][0]


Apple Vision Pro는 혁신적인 공간 컴퓨팅 기술을 소개하는 Apple의 최초의 VisionOS로, 사용자에게 현실적인 경험과 맞춤형 사운드, 뛰어난 해상도의 디스플레이 시스템을 제공하여 혁신적인 시각적 경험을 제공하는 제품이다.
Total Tokens: 3928
Prompt Tokens: 3511
Completion Tokens: 417
Total Cost (USD): 0.01$, Total Cost (Won): 8.24₩
걸린 시간: 6.05초


## Apple 영상에서 궁금한 내용 질문하기